In [ ]:
from multiprocessing import process
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
from ion_channel import IonChannel
import multiprocessing

def worker(file_path, ic : IonChannel, batch_size):
    cnt = 0
    sum = 0
    file_data = []
    
    with open(file_path) as data_file:
        for record in data_file:
            if cnt < batch_size:
                record = record.strip()
                sum += float(record)
                cnt += 1
            else:
                cnt = 0
                file_data.append(sum / batch_size)
                sum = 0
    # Calculate autocorrelation
    _ = file_path.split('/')
    folder = _[2]
    if(not os.path.exists(f"outputs2/{folder}")):
        os.mkdir(f"outputs2/{folder}")
    file_name = _[3].split('.')[0]
    fig, ax = plt.subplots(1, 2, constrained_layout=True)
    fig.set_size_inches(12, 6)
    fig, ax[0] = ic.calculate_autocorrelation_acf(file_data, fig, ax[0], lags=100)
    fig, ax[1] = ic.calculate_autocorrelation_dfa(file_data, fig, ax[1])
    ic.save_figure(fig, folder+'/'+file_name, 'autocorrelation', False)


ic = IonChannel()
file_pattern = os.path.join("data", "normalna_aktywnosc", '*mV', "*.txt")
files = glob.glob(file_pattern)
print(files)
batch_size = 1
stack = files
core_count = multiprocessing.cpu_count()
# Process each file
while len(stack) != 0:
    processes = []
    n = len(stack)
    print(n)
    for i in range(n if n <= core_count else core_count):
        print(i)
        processes.append(multiprocessing.Process(target=worker, args=(stack.pop(), ic, batch_size)))
        processes[i].start()
    for i in range(len(processes)):
        print(len(processes))
        processes[i].join()

['data/normalna_aktywnosc/-20mV/e9.txt']
1
0
1


In [ ]:
import os
import glob
import matplotlib.pyplot as plt
# from monostable_model import calculate_autocorelation_dfa, calculate_autocorelation_acf
import numpy as np
from ion_channel import IonChannel
ic = IonChannel()
# Find all hip files in the data directory
file_pattern = os.path.join("data", "normalna_aktywnosc", '*mV', "e*.txt")
files = glob.glob(file_pattern)
batch_size = 1
# Process each file
for file_path in files:
    cnt = 0
    sum = 0
    file_data = []
    
    with open(file_path) as data_file:
        for record in data_file:
            if cnt < batch_size:
                record = record.strip()
                sum += float(record)
                cnt += 1
            else:
                cnt = 0
                file_data.append(sum / batch_size)
                sum = 0
    
    # Calculate autocorrelation
    _ = file_path.split('/')
    folder = _[2]
    if(not os.path.exists(f"outputs/{folder}")):
        os.mkdir(f"outputs/{folder}")
    file_name = _[3].split('.')[0]
    plt.clf()  
    plt.figure(figsize=(12,8))
    plt.title(folder)
    plt.plot(file_data[:20000])
    # plt.vlines(x=self.breakpoints, ymin=np.min(self.data_transposed[1]), ymax=np.max(self.data_transposed[1]), color='red', linestyle='--')
    plt.xlabel("Records")
    plt.ylabel("Current [pA]")
    plt.show()

array([0.0000e+00, 1.0000e-08, 2.0000e-08, ..., 4.9997e-04, 4.9998e-04,
       4.9999e-04])